## DIA 083: Guardar Predicciones de Usuario en una Base de Datos con Flask + SQLite

Hoy aprenderás a guardar los datos ingresados por el usuario y sus predicciones en una base de datos SQLite usando Flask + SQLAlchemy.

✅ ¿Qué aprenderás hoy?

📌 Conectar Flask con una base de datos SQLite.

📌 Crear un modelo de datos con SQLAlchemy.

📌 Guardar automáticamente cada predicción del usuario en la base de datos.

🛠️ 1. Instalación de Requisitos
bash
Copiar
Editar
pip install flask sqlalchemy
🗂️ 2. Estructura del Proyecto
css
Copiar
Editar
titanic_app/
├── app.py              ← Código principal
├── model.pkl           ← Modelo entrenado
├── db.sqlite3          ← Base de datos SQLite (se crea automáticamente)
└── templates/
    └── index.html      ← Página web
🧩 3. Código Flask (app.py)
python
Copiar
Editar
from flask import Flask, render_template, request
from flask_sqlalchemy import SQLAlchemy
import joblib
import numpy as np
import datetime

# ---------------------------
# Configuración de Flask y la base de datos
# ---------------------------
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite3'
db = SQLAlchemy(app)

# ---------------------------
# Definición del modelo de la base de datos
# ---------------------------
class Registro(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    f1 = db.Column(db.Float)
    f2 = db.Column(db.Float)
    f3 = db.Column(db.Float)
    f4 = db.Column(db.Float)
    prediccion = db.Column(db.String(50))
    fecha = db.Column(db.DateTime, default=datetime.datetime.utcnow)

# ---------------------------
# Cargar el modelo ML
# ---------------------------
modelo = joblib.load("model.pkl")

# ---------------------------
# Ruta principal
# ---------------------------
@app.route("/")
def home():
    return render_template("index.html")

# ---------------------------
# Ruta para predecir y guardar
# ---------------------------
@app.route("/predecir", methods=["POST"])
def predecir():
    try:
        f1 = float(request.form["f1"])
        f2 = float(request.form["f2"])
        f3 = float(request.form["f3"])
        f4 = float(request.form["f4"])

        datos = np.array([[f1, f2, f3, f4]])
        pred = modelo.predict(datos)[0]

        # Guardar en la base de datos
        nuevo_registro = Registro(f1=f1, f2=f2, f3=f3, f4=f4, prediccion=str(pred))
        db.session.add(nuevo_registro)
        db.session.commit()

        return render_template("index.html", resultado=f"✅ Resultado: Clase {pred}")
    except:
        return render_template("index.html", resultado="❌ Error en los datos")

# ---------------------------
# Ejecutar App
# ---------------------------
if __name__ == "__main__":
    db.create_all()  # Crea las tablas si no existen
    app.run(debug=True)
🖼️ 4. Formulario Web (templates/index.html)
(Similar al del Día 82, puede mantenerse igual con Bootstrap)

🔍 Líneas Clave Explicadas
db = SQLAlchemy(app): Conecta Flask con una base de datos SQLite.

class Registro(...): Define la estructura de la tabla para almacenar entradas.

db.create_all(): Crea automáticamente las tablas si no existen.

db.session.add(...): Inserta un nuevo registro en la tabla.

pred = modelo.predict(...): Realiza la predicción con el modelo entrenado.